# Depencies + Imports

In [1]:
!pip install transformers datasets
!pip install torchmetrics
!pip install pytorch-lightning
!pip install openai
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 7.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 9.5 MB/s eta 0:00:00

In [2]:
from math import ceil
import random
import pandas as pd
from torch.utils.data import DataLoader
from torch.optim import Adam 
from torch.nn.functional import cross_entropy
import pytorch_lightning as pl
import json
import numpy as np
import random
from transformers import AutoTokenizer, AutoModel
from torch.nn import Linear, Sigmoid
import torch
import torchmetrics
import requests
import openai
import csv
import umap
from tqdm.auto import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Create Embeddings Loop

In [5]:
#Change paths to downloaded datasets from https://doi.org/10.7910/DVN/TTP7AO
#OpenAI key can be found in presentation. NEVER push it to Github.
GDRIVE_PATH='/content/drive/MyDrive/Hackathon: European Patent Office (EPO)/data/'
EMBEDDING_MODEL='text-embedding-ada-002'
openai.organization_key = 'org-mJggmYzsAWYpllmsKjVpk055'
openai.api_key = ''

# 1 file

In [7]:
CPCs = ['Y02D', 'Y02A', 'Y02B', 'Y02W', 'Y02C', 'Y02E', 'Y02P', 'Y02T', 'not_Y02']
claims_count = 0
for CPC in CPCs:
  with open(GDRIVE_PATH+f'ds_WO_{CPC}_complete_final_new.json', 'r') as file:
      dataset = json.load(file)
      claims_count += len(dataset)


152433


In [11]:
c = 0
with open(GDRIVE_PATH+"relation_extraction_dataset.json", 'r') as file:
      for item in json.load(file): 
        c += len(item[1])
print(c)

4101


In [ ]:
for document_id, claims in tqdm(dataset):
  new_row ={
    "document_id": document_id,
    "embedding": [round(i, 6) for i in openai.Embedding.create(input = claims[:10000], model= EMBEDDING_MODEL)['data'][0]['embedding']]
    }

  with open(f'{GDRIVE_PATH}embeddings/{CPC}.csv', 'a') as f:
        writer = csv.DictWriter(f, fieldnames=['document_id','embedding'])
        writer.writerow(new_row)

  0%|          | 0/7732 [00:00<?, ?it/s]

In [ ]:
CPCs = ['Y02B'] #'Y02C', 'Y02W', 'Y02D', 'not_Y02', 'Y02A', , 'Y02E', 'Y02P', 'Y02T'
for CPC in CPCs:
  with open(GDRIVE_PATH+f'ds_WO_{CPC}_complete_final_new.json', 'r') as file:
      dataset = json.load(file)

  for document_id, claims in tqdm(dataset):
    new_row ={
      "document_id": document_id,
      "embedding": [round(i, 6) for i in openai.Embedding.create(input = claims[:10000], model= EMBEDDING_MODEL)['data'][0]['embedding']]
      }

    with open(f'{GDRIVE_PATH}embeddings/{CPC}.csv', 'a') as f:
          writer = csv.DictWriter(f, fieldnames=['document_id','embedding'])
          writer.writerow(new_row)

  0%|          | 0/15876 [00:00<?, ?it/s]